In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np


In [2]:
df = pd.read_csv("dataset_phishing.csv", sep = ",")


## Encoding para la variable Status

In [4]:
# Crear columnas separadas para "legitimate" y "phishing" con valores 0 o 1
df['legitimate'] = (df['status'] == 'legitimate').astype(int)
df['phishing'] = (df['status'] == 'phishing').astype(int)

In [6]:
n = int(input("Ingrese las filas que desea ver"))
df.head(n)

Ingrese las filas que desea ver 5


,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status,legitimate,phishing
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,45,-1,0,1,1,4,legitimate,1,0
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,0,77,5767,0,0,1,2,phishing,0,1
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,0,14,4004,5828815,0,1,0,phishing,0,1
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,0,62,-1,107721,0,0,3,legitimate,1,0
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,224,8175,8725,0,0,6,legitimate,1,0


## Regresión logistica

### 1.) Procesar los datos

In [9]:
X = df[['length_url', 'nb_dots']].values  # Características
y = df['phishing'].values  # Variable objetivo

In [ ]:
# X = variable independiente
# Y = indica si la instancia es pishing (1) o (0)
# prueba_entrenamiento_split: divide los datos en entrenamiento y prueba
# X_entrenamiento, Y_entrenamiento: despues de dividir los datos contiene el conjunto de entrenamiento de x & y
# X_test, Y_test: despues de dividir los datos contiene el conjunto de prueba de x & y 
x_entrenamiento, x_test, y_entrenamiento, y_test = prueba_entrenamiento_split(X, y, tamanio_test=0.2, Estado_random=42)

##### Colocamos la formula sigmoide de regresión logistica en una función para pasar los parametros

In [ ]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

##### Inicializar parametros w y b

In [ ]:
def inicializarParams(dim):
    w = np.zeros((dim, 1)) #matriz de ceros
    b = 0 #inicializar el sesgo en 0 
    return w, b


In [ ]:
def propagate(w, b, X, y): #w pesos, b sesgo, x matriz de fila representada A, y matriz de fila representada B 
    m = X.shape[0]
    
    # Forward: Se crea el sigmoide, calculando la combinación lineal
    A = sigmoide(np.dot(X, w) + b) 
    # Etropia cruzada: Mide el rendimiento del problema 
    cost = (-1 / m) * np.sum(y * np.log(A) + (1 - y) * np.log(1 - A)) #Al utilizar esta formula obtenemos el coste promedio por todas las instancias
    
    # Backward:  utilizamos los parametros inicializados anteriromente (w y b) calculando los gradientes
    dw = (1 / m) * np.dot(X.T, (A - y)) #actualiza  los pesos en la dirección que reduce el coste
    db = (1 / m) * np.sum(A - y) #gradiante del coste con respecto al sesgo
    
    gradiantes = {"dw": dw, "db": db} #Diccionario con los gradiantes de los parametros utilizados (w y b)
    
    return gradiantes, cost


##### Encontrar los valores optimos para w y b

In [ ]:
# iterations_optimize: Optimizar las iteraciones del conjunto de datos
# learning_rate: Curva de aprendizaje (tamaño de gradiante descendente)
# print_cost: si es true imprime el coste de cada 100 iteraciones
def optimize(w, b, X, y, iterations_optimize, learning_rate, print_cost=False):
    costos = []
    #Bucle para iterations_optimize para calcular los gradiantes actuales (w y b), y actualizarlos
    for i in range(iterations_optimize):
        gradiantes, cost = propagate(w, b, X, y)
        dw = gradiantes["dw"]
        db = gradiantes["db"]
        w = w - learning_rate * dw
        b = b - learning_rate * db
#Si la iteración actual es un múltiplo de 100, el coste se agrega a la lista costos y, si print_cost es True, se imprime el coste
        if i % 100 == 0:
            costos.append(cost)
            if print_cost:
                print(f"Cost after iteration {i}: {cost}")

    params = {"w": w, "b": b}
    gradiantes = {"dw": dw, "db": db}
    
    #Retorna los parametros, costos y gradiantes
    return params, gradiantes, costos


##### Por ultimo se predicen los nuevos datos X

In [2]:
def predict(w, b, X): # x es donde iran los nuevos datos
    m = X.shape[0] #num de ejemplos
    y_prediccion = np.zeros((m, 1)) #Array de ceros donde se almacenaran las predicciones
    w = w.reshape(X.shape[1], 1)
    # calcula la probabilidad de que cada X pertenezca a la clase 1 utilizando sigmoide
    A = sigmoide(np.dot(X, w) + b)
    #asigna una predicción de 1 si la probabilidad es mayor que 0.5, y 0 si no es asi
    for i in range(A.shape[0]):
        y_prediccion[i, 0] = 1 if A[i, 0] > 0.5 else 0
        
    return y_prediccion


## Entrenamiento del modelo

In [ ]:
# Inicializar parámetros
dim = X_train.shape[1]
w, b = initialize_parameters(dim)

# Gradiente descendente
iterations_optimize = 2000  # Número de iteraciones
learning_rate = 0.01
params, gradiantes, costos = optimize(w, b, X_train, y_train, iterations_optimize, learning_rate, print_cost=True)

# Predicciones
w = params["w"]
b = params["b"]
y_pred_train = predict(w, b, X_train)
y_pred_test = predict(w, b, X_test)


Cost after iteration 0: 6338.1378190411515


/tmp/ipykernel_131293/2747025002.py:6: RuntimeWarning: divide by zero encountered in log
  cost = (-1 / m) * np.sum(y * np.log(A) + (1 - y) * np.log(1 - A))
/tmp/ipykernel_131293/2747025002.py:6: RuntimeWarning: invalid value encountered in multiply
  cost = (-1 / m) * np.sum(y * np.log(A) + (1 - y) * np.log(1 - A))


Cost after iteration 100: nan
Cost after iteration 200: nan
Cost after iteration 300: nan


In [ ]:
from sklearn.metrics import accuracy_score

accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print(f"Accuracy on training set: {accuracy_train}")
print(f"Accuracy on test set: {accuracy_test}")


In [ ]:

# Selecciona las instancias de cada clase para el conjunto de entrenamiento
pos = np.where(y_train == 1)  # phishing
neg = np.where(y_train == 0)  # legitimate

# Grafica las características seleccionadas por clase
plt.scatter(X_train[pos, 0], X_train[pos, 1], color='red', label='Phishing', alpha=0.5)
plt.scatter(X_train[neg, 0], X_train[neg, 1], color='green', label='Legitimate', alpha=0.5)

plt.xlabel('Length URL')
plt.ylabel('Number of Dots')
plt.legend()
